In [1]:
import pandas as pd
import tensorflow as tf
import keras_nlp

from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 200)

In [2]:
train_essays = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
train_essays = train_essays.drop(columns=['id','prompt_id'])
train_essays.head()

,text,generated
0,"Cars. Cars have been around since they became famous in the 1900s, when Henry Ford created and built the first ModelT. Cars have played a major role in our every day lives since then. But now, peo...",0
1,"Transportation is a large necessity in most countries worldwide. With no doubt, cars, buses, and other means of transportation make going from place to place easier and faster. However there's alw...",0
2,"""America's love affair with it's vehicles seems to be cooling"" says Elisabeth rosenthal. To understand rosenthal's perspective, it is easier to suggest that America's car usage is decreasing slowl...",0
3,How often do you ride in a car? Do you drive a one or any other motor vehicle to work? The store? To the mall? Have you ever thought about how many people in the world do that exact same thing tra...,0
4,Cars are a wonderful thing. They are perhaps one of the worlds greatest advancements and technologies. Cars get us from point a to point i. That is exactly what we want isnt it? We as humans want ...,0


In [3]:
train_mistral1 = pd.read_csv("/kaggle/input/llm-mistral-7b-instruct-texts/Mistral7B_CME_v1.csv")
train_mistral2 = pd.read_csv("/kaggle/input/llm-mistral-7b-instruct-texts/Mistral7B_CME_v2.csv")
train_mistral3 = pd.read_csv("/kaggle/input/llm-mistral-7b-instruct-texts/Mistral7B_CME_v3.csv")
train_mistral4 = pd.read_csv("/kaggle/input/llm-mistral-7b-instruct-texts/Mistral7B_CME_v4.csv")
train_mistral5 = pd.read_csv("/kaggle/input/llm-mistral-7b-instruct-texts/Mistral7B_CME_v5.csv")
train_mistral6 = pd.read_csv("/kaggle/input/llm-mistral-7b-instruct-texts/Mistral7B_CME_v6.csv")
train_mistral7 = pd.read_csv("/kaggle/input/llm-mistral-7b-instruct-texts/Mistral7B_CME_v7.csv")

train_mistral = pd.concat([train_mistral1, train_mistral2, train_mistral3, train_mistral4, train_mistral5, train_mistral6, train_mistral7])
train_mistral = train_mistral.drop(columns=['prompt_id', 'prompt_name'])
train_mistral.head()

,text,generated
0,"Introduction:\nVenus is one of the most famous planets in our solar system. The planet's extreme temperature, atmospheric pressure, and volcanic activity make it a challenging place to explore. H...",1
1,"Venus, often referred to as Earth’s “sister planet” because of their similarities, is the second planet from the sun. Despite its proximity to Earth, scientists have yet to explore Venus extensiv...",1
2,"Venus, the second planet from the sun, is often referred to as the Earth's ""sister planet"" due to their similarities in size, composition and atmosphere. However, Venus is anything but like Earth...",1
3,"Venus, also known as the Earth's sister planet, is a fascinating topic that has been explored by scientists, researchers, and space agencies for years. The planet's unique characteristics and its...",1
4,"Venus: A Journey into the Future\n\nAs one of the planets closest to Earth, Venus has long held a place in human imagination and mythology. From the ancient Romans who believed it was the goddess...",1


In [4]:
train_daigt = pd.read_csv('/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv')
train_daigt = train_daigt.drop(columns=['prompt_name', 'source', 'RDizzl3_seven']).rename(columns={'label':'generated'})
train_daigt.head()

,text,generated
0,Phones\n\nModern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social me...,0
1,"This essay will explain if drivers should or should not be able to use electronic devices while operating a vehicle. Using a phone while driving can cause very bad wrecks, because they are putting...",0
2,"Driving while the use of cellular devices\n\nToday, most of the society is thoughtless. Especially new drivers, all driver for that matter do not understand the dangers of looking at a cell phone ...",0
3,Phones & Driving\n\nDrivers should not be able to use phones while operating a vehicle. Drivers who used their phone while operating a vehicle are most likely to get into an accident that could be...,0
4,Cell Phone Operation While Driving\n\nThe ability to stay connected to people we know despite distance was originally brought to fruition by the use of letters. This system was found to be rather ...,0


In [5]:
train_data = pd.concat([train_essays, train_mistral, train_daigt])
train_data = train_data.drop_duplicates()
train_data.groupby('generated').count()

,text
generated,
0,27371
1,22397


In [6]:
x_train, x_test, y_train, y_test = train_test_split(train_data['text'], train_data['generated'], test_size=0.1)

In [7]:
preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset(
    "distil_bert_base_en_uncased",
    sequence_length=512)

classifier = keras_nlp.models.DistilBertClassifier.from_preset(
    "distil_bert_base_en_uncased", 
    num_classes=2, 
    preprocessor=preprocessor)

classifier.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    optimizer=tf.keras.optimizers.Adam(5e-5), 
    jit_compile=True)

classifier.fit(x=x_train, y=y_train, batch_size=16, epochs=5, validation_data=(x_test, y_test))

Attaching 'tokenizer.json' from model 'keras/distil_bert/keras/distil_bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/distil_bert/keras/distil_bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.txt' from model 'keras/distil_bert/keras/distil_bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/distil_bert/keras/distil_bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/distil_bert/keras/distil_bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/distil_bert/keras/distil_bert_base_en_uncased/2' to your Kaggle notebook...
/opt/conda/lib/python3.10/site-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(

Epoch 1/5



KeyboardInterrupt



**Final Submission**

In [ ]:
test_essays = pd.read_csv('../input/llm-detect-ai-generated-text/test_essays.csv')
pred = classifier.predict(test_essays['text'])

In [ ]:
submission = pd.DataFrame(test_essays['id'])
submission['generated'] = tf.nn.softmax(pred)[:, 1]
submission.to_csv('submission.csv', index=False)